# OpenAI Agent + Query Engine Experimental Cookbook
In this notebook, we try out the OpenAIAgent across a variety of query engine tools and datasets. We explore how OpenAIAgent can compare/replace existing workflows solved by our retrievers/query engines.

Auto retrieval
Joint SQL and vector search
NOTE: Any Text-to-SQL application should be aware that executing arbitrary SQL queries can be a security risk. It is recommended to take precautions as needed, such as using restricted roles, read-only databases, sandboxing, etc.

AutoRetrieval from a Vector Database
Our existing "auto-retrieval" capabilities (in VectorIndexAutoRetriever) allow an LLM to infer the right query parameters for a vector database - including both the query string and metadata filter.

Since the OpenAI Function API can infer function parameters, we explore its capabilities in performing auto-retrieval here.



https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_query_cookbook/

In [1]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai
%pip install llama-index-readers-wikipedia
%pip install llama-index-vector-stores-pinecone

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.

In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Global Models


In [6]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [7]:
# Using the LlamaDebugHandler to print the trace of the sub questions
# captured by the SUB_QUESTION callback event type
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

In [12]:
import pinecone
api_key = os.environ["PINECONE_API_KEY"]


In [26]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=api_key)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Code\\Github\\LlamaIndex\\venv\\lib\\site-packages\\pinecone_plugins'])
Discovering subpackages in _NamespacePath(['c:\\Code\\Github\\LlamaIndex\\venv\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone


In [31]:
# dimensions are for text-embedding-ada-002
try:
    pc.create_index("quickstart-index", 
                dimension=1536, 
                metric="euclidean",
                spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )
except Exception:
    # most likely index already exists
    pass

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart-index","metric":"euclidean","dimension":1536,"status":{"ready":false,"state":"Initializing"},"host":"quickstart-index-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart-index","metric":"euclidean","dimension":1536,"status":{"ready":false,"state":"Initializing"},"host":"quickstart-index-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart-index","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-index-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart-index","metric":"euclide

In [32]:
pinecone_index = pc.Index("quickstart-index")


DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart-index","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-index-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart-index","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-index-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'


In [ ]:
# Optional: delete data in your pinecone index
pinecone_index.delete(deleteAll=True, namespace="test")

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"code":5,"message":"Namespace not found","details":[]}'
response body: b'{"code":5,"message":"Namespace not found","details":[]}'


NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 01 Nov 2024 08:05:53 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '32', 'x-pinecone-request-id': '6380602307549067385', 'x-envoy-upstream-service-time': '32', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}


In [34]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [35]:
from llama_index.core.schema import TextNode

nodes = [
    TextNode(
        text=(
            "Michael Jordan is a retired professional basketball player,"
            " widely regarded as one of the greatest basketball players of all"
            " time."
        ),
        metadata={
            "category": "Sports",
            "country": "United States",
            "gender": "male",
            "born": 1963,
        },
    ),
    TextNode(
        text=(
            "Angelina Jolie is an American actress, filmmaker, and"
            " humanitarian. She has received numerous awards for her acting"
            " and is known for her philanthropic work."
        ),
        metadata={
            "category": "Entertainment",
            "country": "United States",
            "gender": "female",
            "born": 1975,
        },
    ),
    TextNode(
        text=(
            "Elon Musk is a business magnate, industrial designer, and"
            " engineer. He is the founder, CEO, and lead designer of SpaceX,"
            " Tesla, Inc., Neuralink, and The Boring Company."
        ),
        metadata={
            "category": "Business",
            "country": "United States",
            "gender": "male",
            "born": 1971,
        },
    ),
    TextNode(
        text=(
            "Rihanna is a Barbadian singer, actress, and businesswoman. She"
            " has achieved significant success in the music industry and is"
            " known for her versatile musical style."
        ),
        metadata={
            "category": "Music",
            "country": "Barbados",
            "gender": "female",
            "born": 1988,
        },
    ),
    TextNode(
        text=(
            "Cristiano Ronaldo is a Portuguese professional footballer who is"
            " considered one of the greatest football players of all time. He"
            " has won numerous awards and set multiple records during his"
            " career."
        ),
        metadata={
            "category": "Sports",
            "country": "Portugal",
            "gender": "male",
            "born": 1985,
        },
    ),
]

In [36]:
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="test"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [37]:
index = VectorStoreIndex(nodes, storage_context=storage_context)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x00000180C5431750>, 'json_data': {'input': ['category: Sports country: United States gender: male born: 1963  Michael Jordan is a retired professional basketball player, widely regarded as one of the greatest basketball players of all time.', 'category: Entertainment country: United States gender: female born: 1975  Angelina Jolie is an American actress, filmmaker, and humanitarian. She has received numerous awards for her acting and is known for her philanthropic work.', 'category: Business c

Upserted vectors:   0%|          | 0/5 [00:00<?, ?it/s]

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"upsertedCount":5}'
response body: b'{"upsertedCount":5}'


Upserted vectors: 100%|██████████| 5/5 [00:02<00:00,  2.07it/s]

**********
Trace: index_construction
    |_embedding -> 1.763578 seconds
**********


# Define Function Tool
Here we define the function interface, which is passed to OpenAI to perform auto-retrieval.

We were not able to get OpenAI to work with nested pydantic objects or tuples as arguments, so we converted the metadata filter keys and values into lists for the function API to work with.

In [38]:
# define function tool
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores import (
    VectorStoreInfo,
    MetadataInfo,
    MetadataFilter,
    MetadataFilters,
    FilterCondition,
    FilterOperator,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

# hardcode top k for now
top_k = 3

# define vector store info describing schema of vector store
vector_store_info = VectorStoreInfo(
    content_info="brief biography of celebrities",
    metadata_info=[
        MetadataInfo(
            name="category",
            type="str",
            description=(
                "Category of the celebrity, one of [Sports, Entertainment,"
                " Business, Music]"
            ),
        ),
        MetadataInfo(
            name="country",
            type="str",
            description=(
                "Country of the celebrity, one of [United States, Barbados,"
                " Portugal]"
            ),
        ),
        MetadataInfo(
            name="gender",
            type="str",
            description=("Gender of the celebrity, one of [male, female]"),
        ),
        MetadataInfo(
            name="born",
            type="int",
            description=("Born year of the celebrity, could be any integer"),
        ),
    ],
)

In [40]:
# define pydantic model for auto-retrieval function
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[Any] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names"
            " specified in filter_key_list)"
        ),
    )
    filter_operator_list: List[str] = Field(
        ...,
        description=(
            "Metadata filters conditions (could be one of <, <=, >, >=, ==, !=)"
        ),
    )
    filter_condition: str = Field(
        ...,
        description=("Metadata filters condition values (could be AND or OR)"),
    )


description = f"""\
Use this tool to look up biographical information about celebrities.
The vector database schema is given below:
{vector_store_info.json()}
"""

# Define AutoRetrieve Functions



In [41]:
def auto_retrieve_fn(
    query: str,
    filter_key_list: List[str],
    filter_value_list: List[any],
    filter_operator_list: List[str],
    filter_condition: str,
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    metadata_filters = [
        MetadataFilter(key=k, value=v, operator=op)
        for k, v, op in zip(
            filter_key_list, filter_value_list, filter_operator_list
        )
    ]
    retriever = VectorIndexRetriever(
        index,
        filters=MetadataFilters(
            filters=metadata_filters, condition=filter_condition
        ),
        top_k=top_k,
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)


auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="celebrity_bios",
    description=description,
    fn_schema=AutoRetrieveModel,
)

# Initialize Agent


In [42]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

agent = OpenAIAgent.from_tools(
    [auto_retrieve_tool],
    llm=OpenAI(temperature=0, model="gpt-4o-mini"),
    verbose=True,
)

In [ ]:
response = agent.chat("Tell me about two celebrities from the United States. ")


Added user message to memory: Tell me about two celebrities from the United States. 
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about two celebrities from the United States. '}], 'model': 'gpt-4o-mini', 'stream': False, 'temperature': 0.0, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'celebrity_bios', 'description': 'Use this tool to look up biographical information about celebrities.\nThe vector database schema is given below:\n{"metadata_info":[{"name":"category","type":"str","description":"Catego

In [45]:
print(str(response))

Here are two celebrities born after 1980:

### Rihanna
- **Born**: 1988
- **Profession**: Singer, actress, and businesswoman
- **Notable Achievements**: Rihanna has achieved significant success in the music industry and is recognized for her versatile musical style.

### Cristiano Ronaldo
- **Born**: 1985
- **Profession**: Professional footballer
- **Notable Achievements**: Cristiano Ronaldo is regarded as one of the greatest football players of all time. He has received numerous awards and set multiple records throughout his career.

If you would like more information or details about other celebrities, feel free to ask!


In [ ]:
response = agent.chat("Tell me about two celebrities born after 1980. ")

Added user message to memory: Tell me about two celebrities born after 1980. 
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about two celebrities from the United States. '}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_ihJodUTZWVhTIRmWEPdWmUSD', 'function': {'arguments': '{"query": "celebrity", "filter_key_list": ["country"], "filter_value_list": ["United States"], "filter_operator_list": ["=="], "filter_condition": "AND"}', 'name': 'celebrity_bios'}, 'type': 'function'}, {'id': 'call_104KVdQbH4BNm4M7bNRJMGsx', 'function': {'arguments': '{"query": "celebrity", "filter_key_list": ["country"], "filter_value_list": ["United States"], "filter_operator_list": ["=="], "filter_condition": "AND"}', 'name': 'celebrity_bios'}, 'type': 'function'}]}, {'role': 'tool', 'content': "Error: 1 validation error for MetadataFilters\ncondition\n  Input should be 

In [ ]:
print(str(response))

In [46]:
response = agent.chat(
    "Tell me about few celebrities under category business and born after 1950. "
)


Added user message to memory: Tell me about few celebrities under category business and born after 1950. 
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about two celebrities from the United States. '}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_ihJodUTZWVhTIRmWEPdWmUSD', 'function': {'arguments': '{"query": "celebrity", "filter_key_list": ["country"], "filter_value_list": ["United States"], "filter_operator_list": ["=="], "filter_condition": "AND"}', 'name': 'celebrity_bios'}, 'type': 'function'}, {'id': 'call_104KVdQbH4BNm4M7bNRJMGsx', 'function': {'arguments': '{"query": "celebrity", "filter_key_list": ["country"], "filter_value_list": ["United States"], "filter_operator_list": ["=="], "filter_condition": "AND"}', 'name': 'celebrity_bios'}, 'type': 'function'}]}, {'role': 'tool', 'content': "Error: 1 validation error for MetadataFilters\nc

In [ ]:
print(str(response))

# Joint Text-to-SQL and Semantic Search
This is currently handled by our SQLAutoVectorQueryEngine.

Let's try implementing this by giving our OpenAIAgent access to two query tools: SQL and Vector

NOTE: Any Text-to-SQL application should be aware that executing arbitrary SQL queries can be a security risk. It is recommended to take precautions as needed, such as using restricted roles, read-only databases, sandboxing, etc.


## Load and Index Structured Data
We load sample structured datapoints into a SQL db and index it.

In [47]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)
from llama_index.core import SQLDatabase
from llama_index.core.indices import SQLStructStoreIndex

engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [48]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)

metadata_obj.create_all(engine)

In [49]:
# print tables
metadata_obj.tables.keys()

dict_keys(['city_stats'])

In [50]:
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Berlin", "population": 3645000, "country": "Germany"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [51]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM city_stats")
    print(cursor.fetchall())

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Berlin', 3645000, 'Germany')]


In [52]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [53]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

In [54]:
query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
)

# Load and Index Unstructured Data
We load unstructured data into a vector index backed by Pinecone

In [55]:
# install wikipedia python package
!pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl


In [56]:
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

In [57]:
cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

In [58]:
# define pinecone index
from pinecone import Pinecone, ServerlessSpec
import os

api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=api_key)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['c:\\Code\\Github\\LlamaIndex\\venv\\lib\\site-packages\\pinecone_plugins'])
Discovering subpackages in _NamespacePath(['c:\\Code\\Github\\LlamaIndex\\venv\\lib\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone


In [59]:
# dimensions are for text-embedding-ada-002
try:
    pc.create_index("quickstart", 
                dimension=1536, 
                metric="euclidean",
                spec=ServerlessSpec(cloud="aws",region="us-east-1"),
    )
except Exception:
    # most likely index already exists
    pass

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":false,"state":"Initializing"},"host":"quickstart-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":false,"state":"Initializing"},"host":"quickstart-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":tru

In [ ]:
# vector_store = PineconeVectorStore(
#     pinecone_index=pinecone_index, namespace="test"
# )
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [63]:
pinecone_index = pc.Index("quickstart")


Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /indexes/quickstart
DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'
response body: b'{"name":"quickstart","metric":"euclidean","dimension":1536,"status":{"ready":true,"state":"Ready"},"host":"quickstart-sohrk74.svc.aped-4627-b74a.pinecone.io","spec":{"serverless":{"region":"us-east-1","cloud":"aws"}},"deletion_protection":"disabled"}'


In [64]:
from llama_index.core import Settings
from llama_index.core import StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI

# define node parser and LLM
Settings.llm = OpenAI(temperature=0, model="gpt-4o-mini")
Settings.node_parser = TokenTextSplitter(chunk_size=1024)

# define pinecone vector index
vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wiki_cities"
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex([], storage_context=storage_context)

**********
Trace: index_construction
**********


In [65]:
# Insert documents into vector index
# Each document has metadata of the city attached
for city, wiki_doc in zip(cities, wiki_docs):
    nodes = Settings.node_parser.get_nodes_from_documents([wiki_doc])
    # add metadata to each node
    for node in nodes:
        node.metadata = {"title": city}
    vector_index.insert_nodes(nodes)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Toronto is the most populous city in Canada and...
> Adding chunk: Toronto is the most populous city in Canada and...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Hill) on the banks of the Rouge River and Teiai...
> Adding chunk: Hill) on the banks of the Rouge River and Teiai...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: city was also the location of Government House,...
> Adding chunk: city was also the location of Government House,...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: doubled to two million by 1971. Following the e...
> Adding chunk: doubled to two million by 1971. Following the e...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: as one of 16 cities in North America (and one o...
> Adding chunk: as one of 16 cities in North America (and one o...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The former mouth drained into a wetlan

Upserted vectors:   0%|          | 0/18 [00:00<?, ?it/s]

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"upsertedCount":18}'
response body: b'{"upsertedCount":18}'


Upserted vectors: 100%|██████████| 18/18 [00:03<00:00,  4.64it/s]

**********
Trace: insert_nodes
    |_embedding -> 1.450323 seconds
**********


...

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Tokyo, officially the Tokyo Metropolis, is the ...
> Adding chunk: Tokyo, officially the Tokyo Metropolis, is the ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: the city was changed to Tokyo (東京, from 東 tō "e...
> Adding chunk: the city was changed to Tokyo (東京, from 東 tō "e...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: which is now part of the Tokaido line. The 1870...
> Adding chunk: which is now part of the Tokaido line. The 1870...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: the war, Tokyo became the base from which the A...
> Adding chunk: the war, Tokyo became the base from which the A...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: July 23 to August 8, 2021, as a result of the C...
> Adding chunk: July 23 to August 8, 2021, as a result of the C...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: area.
Tokyo has numerous outlying isla

Upserted vectors:   0%|          | 0/17 [00:00<?, ?it/s]

DEBUG:pinecone.core.openapi.shared.rest:response body: b'{"upsertedCount":17}'
response body: b'{"upsertedCount":17}'


Upserted vectors: 100%|██████████| 17/17 [00:00<00:00, 19.59it/s]

**********
Trace: insert_nodes
    |_embedding -> 1.755061 seconds
**********


In [66]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.tools import QueryEngineTool


vector_store_info = VectorStoreInfo(
    content_info="articles about different cities",
    metadata_info=[
        MetadataInfo(
            name="title", type="str", description="The name of the city"
        ),
    ],
)
vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever,
)

In [67]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table containing: city_stats, containing the population/country of"
        " each city"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=retriever_query_engine,
    name="vector_tool",
    description=(
        f"Useful for answering semantic questions about different cities"
    ),
)

# Initialize Agent


In [68]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

agent = OpenAIAgent.from_tools(
    [sql_tool, vector_tool],
    llm=OpenAI(temperature=0, model="gpt-4o-mini"),
    verbose=True,
)

In [69]:
# NOTE: gpt-3.5 gives the wrong answer, but gpt-4 is able to reason over both loops
response = agent.chat(
    "Tell me about the arts and culture of the city with the highest"
    " population"
)
print(str(response))

Added user message to memory: Tell me about the arts and culture of the city with the highest population
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
load_verify_locations cafile='C:\\Code\\Github\\LlamaIndex\\venv\\Library\\ssl\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about the arts and culture of the city with the highest population'}], 'model': 'gpt-4o-mini', 'stream': False, 'temperature': 0.0, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'sql_tool', 'description': 'Useful for translating a natural language query into a SQL query over a table containing: city_stats, containing the population/country of each city', 

In [70]:
response = agent.chat("Tell me about the history of Berlin")
print(str(response))

Added user message to memory: Tell me about the history of Berlin
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about the arts and culture of the city with the highest population'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_YxFkQKiQu3UXOKQsMUn4lyoh', 'function': {'arguments': '{"input":"SELECT city, population FROM city_stats ORDER BY population DESC LIMIT 1;"}', 'name': 'sql_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The city with the highest population is Tokyo, which has a population of 13,960,000.', 'name': 'sql_tool', 'tool_call_id': 'call_YxFkQKiQu3UXOKQsMUn4lyoh'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_toJOhHqvmWkO8FwyobT1eQjQ', 'function': {'arguments': '{"input":"arts and culture of Tokyo"}', 'name': 'vector_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': "Tokyo boasts a rich

In [71]:
response = agent.chat(
    "Can you give me the country corresponding to each city?"
)
print(str(response))

Added user message to memory: Can you give me the country corresponding to each city?
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Tell me about the arts and culture of the city with the highest population'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_YxFkQKiQu3UXOKQsMUn4lyoh', 'function': {'arguments': '{"input":"SELECT city, population FROM city_stats ORDER BY population DESC LIMIT 1;"}', 'name': 'sql_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 'The city with the highest population is Tokyo, which has a population of 13,960,000.', 'name': 'sql_tool', 'tool_call_id': 'call_YxFkQKiQu3UXOKQsMUn4lyoh'}, {'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'call_toJOhHqvmWkO8FwyobT1eQjQ', 'function': {'arguments': '{"input":"arts and culture of Tokyo"}', 'name': 'vector_tool'}, 'type': 'function'}]}, {'role': 'tool', 'content': 